##  Feature Engineering

To improve predictive performance, we add **domain-driven features** that directly capture known failure mechanisms from the AI4I 2020 dataset:

- **Temperature difference (`temp_diff`)**  
  - Formula: `process_temperature - air_temperature`  
  - Reason: Machines fail (HDF = Heat Dissipation Failure) when heat cannot escape. A smaller temperature difference means poor cooling.  

- **Power (`power`)**  
  - Formula: `torque * rotational_speed`  
  - Reason: Power Failures (PWF) happen when torque × speed is outside safe operating limits. This derived feature makes the relationship explicit.  

- **Wear × Torque (`wear_torque`)**  
  - Formula: `tool_wear * torque`  
  - Reason: Overstrain Failures (OSF) occur when tools under heavy wear are also subjected to high torque → tool breakage.  

- **Normalized Wear (`norm_wear`)**  
  - Formula: `tool_wear / wear_limit_by_product_type`  
  - Reason: Each product type (Low(L)/Medium(M)/High(H)) allows different wear limits. Normalizing wear makes tool usage comparable across product types.  

- **One-Hot Encoded Product Type (`type_M`, `type_H`)**  
  - Reason: Product type influences tool wear (H > M > L). Encoding product categories lets the model capture this effect.  

---

👉 These features are designed to align with the **five failure modes**:  

- **TWF** = Tool Wear Failure  
- **HDF** = Heat Dissipation Failure  
- **PWF** = Power Failure  
- **OSF** = Overstrain Failure  
- **RNF** = Random Failure  

**Takeaway:**  
Good feature engineering embeds domain knowledge into the data. By mirroring the actual failure mechanics (TWF, HDF, PWF, OSF, RNF), we make patterns learnable with simpler models and significantly improve both accuracy and interpretability.

In [21]:
import pandas as pd
df = pd.read_csv("/Users/swetha/predictive-maintenance-etl-ml/data/ai4i2020.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [22]:
# Temperature difference (for HDF detection)
df["temp_diff"] = df["process_temperature_[k]"] - df["air_temperature_[k]"]

# Power = torque * rotational speed (for PWF detection)
df["power"] = df["torque_[nm]"] * df["rotational_speed_[rpm]"]

# Wear × Torque interaction (for OSF detection)
df["wear_torque"] = df["tool_wear_[min]"] * df["torque_[nm]"]

# Normalized wear by product type
wear_limits = {"l": 240, "m": 250, "h": 260}
df["norm_wear"] = df.apply(lambda x: x["tool_wear_[min]"] / wear_limits[x["type"].lower()], axis=1)

# One-hot encode product type
df = pd.get_dummies(df, columns=["type"], drop_first=True)

In [23]:
df.head()
df.describe()

,udi,air_temperature_[k],process_temperature_[k],rotational_speed_[rpm],torque_[nm],tool_wear_[min],machine_failure,twf,hdf,pwf,osf,rnf,temp_diff,power,wear_torque,norm_wear
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,300.004930,310.005560,1538.776100,39.986910,107.951000,0.033900,0.004600,0.011500,0.009500,0.009800,0.00190,10.000630,59967.147040,4314.664550,0.440984
std,2886.89568,2.000259,1.483734,179.284096,9.968934,63.654147,0.180981,0.067671,0.106625,0.097009,0.098514,0.04355,1.001094,10193.093881,2826.567692,0.260542
min,1.00000,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,7.600000,10966.800000,0.000000,0.000000
25%,2500.75000,298.300000,308.800000,1423.000000,33.200000,53.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,9.300000,53105.400000,1963.650000,0.216000
50%,5000.50000,300.100000,310.100000,1503.000000,40.100000,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,9.800000,59883.900000,4012.950000,0.441667
75%,7500.25000,301.500000,311.100000,1612.000000,46.800000,162.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,11.000000,66873.750000,6279.000000,0.662500
max,10000.00000,304.500000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,12.100000,99980.400000,16497.000000,1.045833


In [24]:
df.to_csv("/Users/swetha/predictive-maintenance-etl-ml/data/ai4i2020_featured.csv", index=False)